In [ ]:
import numpy as np 
import pandas as pd 
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from sklearn.model_selection import train_test_split
from datetime import datetime
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, cross_val_score

import matplotlib 
from matplotlib import pyplot as plt 

from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from sklearn import tree

In [ ]:
def load_data():
    df_wind = pd.read_csv("Processed_data.csv",index_col= 0)
    prices_df = pd.read_csv("Prices_processed.csv",index_col =0).round(1)

    #train, test_prices = train_test_split(prices, test_size=0.25)
    
    Wind_data = df_wind.Actual

    #For calculating whole, 
    prices_train, prices_test, wind_train, wind_test = train_test_split(prices_df, Wind_data, test_size=0.25, shuffle = False)

    P_DA_vals,obj_val,p_E_up_list,p_E_down_list = Optimization_wind(Wind_data,prices_df)

    P_DA_vals = pd.Series(P_DA_vals)

    p_E_up_arr = np.array(p_E_up_list)
    p_E_down_arr = np.array(p_E_down_list)


    #obj_vals_hour_arr = np.array(list(obj_vals_hour.items()))[:,1].flatten()
    vals_df = pd.concat([prices_df,df_wind["mean_wind_speed"]],axis=1).drop(columns="HourUTC")
    vals_df = prices_df.drop(columns="HourUTC")
    
    #x_train_pda, x_test_pda, y_train_pda, y_test_pda  = train_test_split(vals_df, P_DA_vals, test_size=0.25, shuffle = False)
    #scaler = MinMaxScaler()
    #x_train_pda_scaled = pd.DataFrame(scaler.fit_transform(x_train_pda.values), columns=vals_df.columns, index=x_train_pda.index)
    #x_test_pda_scaled = pd.DataFrame(scaler.transform(x_test_pda.values), columns=vals_df.columns, index=x_test_pda.index)

    scaler = MinMaxScaler()
    vals_df_scaled = pd.DataFrame(scaler.fit_transform(vals_df.values), columns=vals_df.columns, index=vals_df.index)

    x_train_pda_scaled , x_test_pda_scaled, y_train_pda, y_test_pda  = train_test_split(vals_df_scaled, P_DA_vals, test_size=0.25, shuffle = False)
    
    return x_train_pda_scaled , x_test_pda_scaled, y_train_pda, y_test_pda ,prices_train, prices_test, wind_train, wind_test ,prices_df


In [ ]:
def Optimization_wind(p_real,prices):
    p_real = p_real.to_numpy()
    Time = len(p_real)
    T = range(Time)

    DA_prices = prices["SpotPriceEUR"].to_numpy()+0.0001
    Up_prices = prices["Up-regulating_price"].to_numpy()
    Down_prices = prices["Down-regulating_price"].to_numpy()
    Capacity = 1 

    psi_up= Up_prices -DA_prices
    psi_down = DA_prices- Down_prices

    model_opt = gp.Model("Step1_A")
    p_DA_A = model_opt.addVars(Time,lb=0)
    p_delt = model_opt.addVars(Time,lb=-gp.GRB.INFINITY,ub=gp.GRB.INFINITY)

    p_E_up = model_opt.addVars(Time,lb =-1,ub=0)
    p_E_down = model_opt.addVars(Time,lb =0,ub=1)

    model_opt.setObjective(
    gp.quicksum(p_DA_A[t]*DA_prices[t] 
            + (Up_prices[t] *  p_E_up[t] + Down_prices[t] *  p_E_down[t]) for t in T )
            ,gp.GRB.MAXIMIZE)

    
    model_opt.addConstrs(
    (p_DA_A[t] <= Capacity
    for t in T
    )
    )

    model_opt.addConstrs(
    (
    p_delt[t] == p_real[t]-p_DA_A[t]
    for t in T
    )
    )

    model_opt.addConstrs(
    (
    p_delt[t] == p_E_up[t] + p_E_down[t]
    for t in T
    )
    )
    

    model_opt.setParam('OutputFlag', False )
    model_opt.optimize()
    if model_opt.status == GRB.OPTIMAL:
        obj_val =  model_opt.objVal 
        print("Revenue Optimization =", "{:.2f}".format(model_opt.objVal))
        #print("Offering Strategy =", ["{:.2f}".format(p_DA_A[t].x) for t in T])
        #print("Imbalance Power =", ["{:.2f}".format(p_delt[t].x) for t in T])
        #print("Real Power  =", ["{:.2f}".format(p_real[t]) for t in T])
        P_DA = [p_DA_A[t].x for t in T]
        obj_vals_hour ={t: p_DA_A[t].x*DA_prices[t] - (Up_prices[t] *  p_E_up[t].x + Down_prices[t] *  p_E_down[t].x) for t in T }

        p_E_up_list =[p_E_up[t].x for t in T]
        p_E_down_list = [p_E_down[t].x for t in T]


    else:
        print("Optimization was not successful")

    return P_DA,obj_val,p_E_up_list,p_E_down_list

In [ ]:
def Optimization_PDA(Day_ahead,p_real,prices):
    p_real = p_real.to_numpy()

    Time = len(Day_ahead)
    T = range(Time)


    DA_prices = prices["SpotPriceEUR"].to_numpy()
    Up_prices = prices["Up-regulating_price"].to_numpy()
    Down_prices = prices["Down-regulating_price"].to_numpy()

    Capacity = 1 

    psi_up= Up_prices -DA_prices
    psi_down = DA_prices- Down_prices

    Model_2_PDA = gp.Model("Model_2_PDA")
    p_DA_A = Model_2_PDA.addVars(Time,lb=0)
    p_delt = Model_2_PDA.addVars(Time,lb=-gp.GRB.INFINITY,ub=gp.GRB.INFINITY)

    p_E_up = Model_2_PDA.addVars(Time,lb =-1,ub=0)
    p_E_down = Model_2_PDA.addVars(Time,lb =0,ub=1)

    Model_2_PDA.setObjective(
    gp.quicksum(Day_ahead[t]*DA_prices[t] 
            + (Up_prices[t] *  p_E_up[t] + Down_prices[t] *  p_E_down[t]) for t in T )
            ,gp.GRB.MAXIMIZE)

    Model_2_PDA.addConstrs(
    (
    p_delt[t] == p_real[t]-Day_ahead[t]
    for t in T
    )
    )
    Model_2_PDA.addConstrs(
    (
    p_delt[t] == p_E_up[t] + p_E_down[t]
    for t in T
    )
    )
    Model_2_PDA.setParam('OutputFlag', False )
    Model_2_PDA.optimize()
    if Model_2_PDA.status == GRB.OPTIMAL:
        obj_val =  Model_2_PDA.objVal 
        print("Revenue opt with fixed wind   =", "{:.2f}".format(Model_2_PDA.objVal))
       # print("Offering Strategy =", ["{:.2f}".format(Day_ahead[t]) for t in T])
       # print("Imbalance Power =", ["{:.2f}".format(p_delt[t].x) for t in T])
       # print("Upbalance =", ["{:.2f}".format(p_E_up[t].x) for t in T])
       # print("Downbalance =", ["{:.2f}".format(p_E_down[t].x) for t in T])
       # print("Real Power  =", ["{:.2f}".format(p_real[t]) for t in T])
        P_delt = [p_delt[t].x for t in T]
        obj_vals_hour ={t: Day_ahead[t]*DA_prices[t] - (Up_prices[t] *  p_E_up[t].x + Down_prices[t] *  p_E_down[t].x) for t in T }


    else:
        print("Optimization was not successful")

    return obj_val, P_delt

In [ ]:
def rfr_regression_PDA(x_train_pda_scaled , x_test_pda_scaled, y_train_pda, y_test_pda):
    
    rfr = RandomForestRegressor()
    param_grid = {
        'max_depth': [3,5,8,10],
        'criterion' :['squared_error'],
        'n_estimators': [10,20,40,]
    }
    
    rfrCV  = GridSearchCV(estimator = rfr, cv=TimeSeriesSplit(n_splits=5),param_grid = param_grid, return_train_score = True)
    rfrCV.fit(x_train_pda_scaled, y_train_pda)
    best_rfr = rfrCV.best_estimator_
    #rfr.fit(X_train, y_train)
    
    #Use the trained regression model to make predictions on the test data.
    y_pred_PDA_rfr = best_rfr.predict(x_test_pda_scaled)
    
    y_pred_PDA_rfr[y_pred_PDA_rfr <0] =0
    
    #check error 
    print('MSE:',mean_squared_error(y_test_pda,y_pred_PDA_rfr))
    print('R^2:', r2_score(y_test_pda, y_pred_PDA_rfr, force_finite=False))
    print('Score:',best_rfr.score(x_train_pda_scaled, y_train_pda))
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test_pda,y_pred_PDA_rfr, c='b', label='Fitted Data',alpha=0.4)
    plt.plot([min(y_test_pda), max(y_test_pda)], [min(y_test_pda), max(y_test_pda)], 'k--', lw=2, label='Ideal Fit')
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.title('True vs. Predicted Values')
    plt.legend()
    plt.show()
    
    
    plt.figure(figsize=(12, 5))
    plt.scatter(x_test_pda_scaled.index,y_test_pda, c='b', label='True Data',alpha=0.3)
    plt.scatter(x_test_pda_scaled.index,y_pred_PDA_rfr, c='g', label='Fitted Data',alpha=0.25)
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.title('True vs. Predicted Values')
    plt.legend()
    plt.show()
    
    return y_pred_PDA_rfr


In [ ]:
def reg_L1(x_train_pda_scaled , x_test_pda_scaled, y_train_pda, y_test_pda):
    
    # Define the Lasso Regression model
    lasso_model = Lasso()
    
    # Define a range of alpha values to test
    alphas = np.logspace(-6, 6, 13) 
    
    #Define the 5 fold time-series cross validation
    
    param_grid = { "fit_intercept" :[True,False],  "alpha": alphas}
    
    lassoCV  = GridSearchCV(estimator = lasso_model, cv=TimeSeriesSplit(n_splits=5),param_grid = param_grid, return_train_score = True)
    lassoCV.fit(x_train_pda_scaled, y_train_pda)
    best_lassoCV = lassoCV.best_estimator_
    
  
    y_pred_cv = best_lassoCV.predict(x_test_pda_scaled)
    print('MSE:',mean_squared_error(y_test_pda,y_pred_cv))
    print('R^2:', r2_score(y_test_pda, y_pred_cv, force_finite=False))
    print('Score:',best_lassoCV.score(x_train_pda_scaled, y_train_pda))
    
    
    
    
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test_pda,y_pred_cv, c='b', label='Fitted Data')
    plt.plot([min(y_test_pda), max(y_test_pda)], [min(y_test_pda), max(y_test_pda)], 'k--', lw=2, label='Ideal Fit')
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.title('True vs. Predicted Values')
    plt.legend()
    plt.show()
    
    plt.figure(figsize=(15, 5))
    plt.scatter(np.arange(0,len(y_test_pda)),y_test_pda, c='b', label='True Data',alpha=0.5)
    plt.scatter(np.arange(0,len(y_test_pda)),y_pred_cv, c='g', label='Fitted Data',alpha=0.4)
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.title('True vs. Predicted Values')
    plt.legend()
    plt.show()
    
    return y_pred_cv

In [ ]:
def reg_L2(x_train_pda_scaled , x_test_pda_scaled, y_train_pda, y_test_pda):
    
    
    # Define the Lasso Regression model
    ridge_model = Ridge()
    
    # Define a range of alpha values to test
    alphas = np.logspace(-6, 6, 13) 
    
    #Define the 5 fold time-series cross validation
    
    param_grid = { "fit_intercept" :[True,False],  "alpha": alphas}
    
    ridgeCV  = GridSearchCV(estimator = ridge_model, cv=TimeSeriesSplit(n_splits=5),param_grid = param_grid, return_train_score = True)
    ridgeCV.fit(x_train_pda_scaled, y_train_pda)
    best_ridgeCV = ridgeCV.best_estimator_
  
    y_pred_cv = best_ridgeCV.predict(x_test_pda_scaled)

    print('MSE:',mean_squared_error(y_test_pda,y_pred_cv))
    print('R^2:', r2_score(y_test_pda, y_pred_cv, force_finite=False))
    print('Score:',best_ridgeCV.score(x_train_pda_scaled, y_train_pda))
    
    
    
    
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test_pda,y_pred_cv, c='b', label='Fitted Data')
    plt.plot([min(y_test_pda), max(y_test_pda)], [min(y_test_pda), max(y_test_pda)], 'k--', lw=2, label='Ideal Fit')
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.title('True vs. Predicted Values')
    plt.legend()
    plt.show()
    
    plt.figure(figsize=(15, 5))
    plt.scatter(np.arange(0,len(y_test_pda)),y_test_pda, c='b', label='True Data',alpha=0.5)
    plt.scatter(np.arange(0,len(y_test_pda)),y_pred_cv, c='g', label='Fitted Data',alpha=0.4)
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.title('True vs. Predicted Values')
    plt.legend()
    plt.show()

    return y_pred_cv

In [ ]:
def clustering_prices(x_train_pda_scaled , x_test_pda_scaled, y_train_pda, y_test_pda ,prices):
    
    """
    
    We manually cluster the prices based on price, and then turn those clustered
    values into their own data set, that we can then use regerssion on
    
    """
    
    
    Time = len(prices)
    T = range(Time)
    
    DA_prices = prices["SpotPriceEUR"].to_numpy()
    Up_prices = prices["Up-regulating_price"].to_numpy()
    Down_prices = prices["Down-regulating_price"].to_numpy()
    
    p_DA_clust = np.empty(Time)
    
 
    for t in T:
        if DA_prices[t] < Down_prices[t] :
            p_DA_clust[t] = 1
        elif DA_prices[t] >= Up_prices[t] :
            p_DA_clust[t] = 2
        else:
            p_DA_clust[t] = 0
    p_DA_clust = pd.Series(p_DA_clust)
    
    pDA_train_clust,pDA_test_clust,= train_test_split(p_DA_clust)
    
    u_labels_train = np.unique(pDA_train_clust).astype(int)
    u_labels_test = np.unique(pDA_test_clust).astype(int)

    # Create a list of indices at which to split the array
    split_indices_train = []
    split_indices_test = []
    clusted_dfs_train = []
    clustered_y_train = []
    clusted_dfs_test = []
    clustered_y_test = []
    
    fig, axes = plt.subplots(3, sharex=True)
    #Cluster of time series 
    for (i,ax) in enumerate(axes.flatten()):
        ax.set_title("Cluster " + str(i+1))
        ax.set_ylabel("Day Ahead Bid")
        ax.set_xlabel("Time")
        ax.plot(x_test_pda_scaled.index, y_test_pda, 'r-',label = "orig",alpha = 0.4,linewidth = 1.5)
        ax.plot(x_test_pda_scaled[p_DA_clust == i].index, y_test_pda[p_DA_clust == i], 'g-',label = "Cluster",alpha =0.6, linewidth = 3)
    handles, labels = ax.get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center')
    fig.suptitle('Wind production seperated into clusters vs time :Test Set ', fontsize=16)
    fig.tight_layout()
    plt.show()
     
     
     #save each train cluster into their own dataframe  and to list 
    for i in range(len(u_labels_train)):
        split_indices_train.append(np.where(pDA_train_clust == i)[0])
        clusted_dfs_train.append( x_train_pda_scaled.iloc[split_indices_train[i]].reset_index(drop=True))
        clustered_y_train.append(y_train_pda.iloc[split_indices_train[i]].reset_index(drop=True))
        
        
     #save each test cluster into their own dataframe  and to list 
    for i in range(len(u_labels_test)):
       
        split_indices_test.append(np.where(pDA_test_clust == u_labels_test[i])[0])
        clusted_dfs_test.append(x_test_pda_scaled.iloc[split_indices_test[i]])
        clustered_y_test.append(y_test_pda.iloc[split_indices_test[i]])
    
    
    #Local regression 
    
    
    return u_labels_train, u_labels_test, clusted_dfs_train, clustered_y_train, clusted_dfs_test, clustered_y_test

In [ ]:
def clusted_rfr(label_train,u_labels_test,clusted_dfs_train,clustered_y_train,clusted_dfs_test,clustered_y_test): 
    
    """
    Input training data seperated into per cluster
    A Random Forest regression is trained on the clusted data 
    
    When all clusters have been trained and predicted with its best estimator, 
    all the predictions are recombined into one set with their orignial index so time is maintained
    
    This model has an issue and computes the value wrong somewhere
    
    return: recombined test data, best MSE of all models, best r-score of all models 
    """
    
    #Estimator paramters to be checked for best 
    rfr = RandomForestRegressor()
    param_grid = {
        'max_depth': [3,5,8,10],
        'criterion' :['squared_error'],
        'n_estimators': [10,20,40,]
    } 
    
    
    y_pred_list = []
    y_pred_df = []
    
    r_scores = np.empty(len(u_labels_test))
    mse_scores = np.empty(len(u_labels_test))
    fig, axes = plt.subplots(3, sharex=True)
    #Each cluster is trained individually and best is kept 
    for (k,ax) in enumerate(axes.flatten()):

        x_train = clusted_dfs_train[k]
        y_train = clustered_y_train[k]
        
        x_test =clusted_dfs_test[k]
        y_test = clustered_y_test[k]
        
        
        rfrCV  = GridSearchCV(estimator = rfr, cv=TimeSeriesSplit(n_splits=5),param_grid = param_grid, return_train_score = True)
        rfrCV.fit(x_train, y_train)
        best_rfr = rfrCV.best_estimator_
        
        y_pred_sk = best_rfr.predict(x_test)
        
        #Save to list 
        y_pred_list.append(y_pred_sk)
        y_pred_df.append(pd.DataFrame(y_pred_sk,index=y_test.index))
        
        print('MSE:',mean_squared_error(y_test,y_pred_sk))
        print('R^2:', r2_score(y_test, y_pred_sk, force_finite=False))
        
        r_scores[k] = r2_score(y_test, y_pred_sk, force_finite=False)
        mse_scores[k] = mean_squared_error(y_test,y_pred_sk)
        
        
        ax.set_title("Cluster " + str(k+1))
        ax.set_ylabel("Wind Power")
        ax.set_xlabel("Time")
        ax.plot(x_test.index, y_test, 'r-',label = "Orig",alpha=0.7)
        ax.plot(x_test.index, y_pred_sk, 'g-',label = "Cluster",alpha=0.6)
   
    
    handles, labels = ax.get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center')
    fig.suptitle('Random forrest on Wind Forecast: Test Set', fontsize=16)
    fig.tight_layout()
    plt.show()
    
    #all clusters recombined , index maintained 
    Clust_rfr_combined = pd.concat(y_pred_df).squeeze()
    
    return Clust_rfr_combined,r_scores,mse_scores

In [ ]:
def decision_tree(x_train_pda_scaled , x_test_pda_scaled, y_train_pda, y_test_pda):
    
    clf = tree.DecisionTreeRegressor()
    
    params = { 'max_depth' : [1,2,3], 
        }
    
    grid_clf = GridSearchCV(clf, param_grid = params, cv=TimeSeriesSplit(n_splits=5))
    grid_clf.fit(x_train_pda_scaled,y_train_pda)
    
    best_clf = grid_clf.best_estimator_
    
  
    y_pred_clf = best_clf.predict(x_test_pda_scaled)
    print('MSE:',mean_squared_error(y_test_pda,y_pred_clf))
    print('R^2:', r2_score(y_test_pda, y_pred_clf, force_finite=False))
    print('Score:',best_clf.score(x_train_pda_scaled, y_train_pda))
    
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test_pda,y_pred_clf, c='b', label='Fitted Data')
    plt.plot([min(y_test_pda), max(y_test_pda)], [min(y_test_pda), max(y_test_pda)], 'k--', lw=2, label='Ideal Fit')
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.title('True vs. Predicted Values')
    plt.legend()
    plt.show()
    
    plt.figure(figsize=(15, 5))
    plt.scatter(np.arange(0,len(y_test_pda)),y_test_pda, c='b', label='True Data',alpha=0.5)
    plt.scatter(np.arange(0,len(y_test_pda)),y_pred_clf, c='g', label='Fitted Data',alpha=0.4)
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.title('True vs. Predicted Values')
    plt.legend()
    plt.show()
    
    
    return y_pred_clf

In [ ]:
def balancing_revenue(P_DA_bid, p_real, prices):
    
    """
    
    Input: 
    Bids made for Day ahead market
    The actual wind production 
    The prices for test set
    
    We calculate the realization of our day ahead bid in the market
    
    Output: 
        
    Revenue: Our actual money gained
    
    Up and down regulation made by our Day ahead bid
    
    """
    
    Time = len(P_DA_bid)
    T = range(Time)
    p_real = p_real.to_numpy()
    
    if isinstance(P_DA_bid,pd.core.series.Series):
        P_DA_bid = P_DA_bid.to_numpy()
    

    DA_prices = prices["SpotPriceEUR"].to_numpy()
    Up_prices = prices["Up-regulating_price"].to_numpy()
    Down_prices = prices["Down-regulating_price"].to_numpy()
    
    p_up = np.empty(Time)
    p_down = np.empty(Time)
 
    for t in T:
        if p_real[t] <= P_DA_bid[t] :
            p_up[t] = p_real[t] -P_DA_bid[t]
        else:
            p_up[t] = 0
        if p_real[t] > P_DA_bid[t] :
            p_down[t] = p_real[t] - P_DA_bid[t]
        else:
            p_down[t] = 0
    Revenue = np.dot(P_DA_bid,DA_prices) + np.dot(p_up,Up_prices) + np.dot(p_down,Down_prices)
    
    return Revenue, p_up,p_down


## Main for 2nd model, uncomment to run 

In [ ]:
#Get data for use
x_train_pda_scaled , x_test_pda_scaled, y_train_pda, y_test_pda ,prices_train ,\
prices_test, wind_train, wind_test, prices = load_data()

#y_pred_tree = decision_tree(x_train_pda_scaled , x_test_pda_scaled, y_train_pda, y_test_pda)

#Normal regression
y_pred_PDA_reg = regression_PDA(x_train_pda_scaled , x_test_pda_scaled, y_train_pda, y_test_pda)

#lasso regression PDA 
y_pred_PDA_L1 = reg_L1(x_train_pda_scaled , x_test_pda_scaled, y_train_pda, y_test_pda)

#Ridge regression PDA 
y_pred_PDA_L2 = reg_L2(x_train_pda_scaled , x_test_pda_scaled, y_train_pda, y_test_pda)

#Random forest regressor prediction on Day ahead bidding 
y_pred_PDA_rfr = rfr_regression_PDA(x_train_pda_scaled , x_test_pda_scaled, y_train_pda, y_test_pda)



#Clustered rfr
u_labels_train, u_labels_test, clusted_dfs_train, clustered_y_train, clusted_dfs_test, clustered_y_test = clustering_prices(x_train_pda_scaled , x_test_pda_scaled, y_train_pda, y_test_pda ,prices)
clust_rfr_pred,r_scores,mse_scores = clusted_rfr(u_labels_train, u_labels_test, clusted_dfs_train, clustered_y_train, clusted_dfs_test, clustered_y_test)



#Fixing values to between [0,1]
y_pred_PDA_reg_copy = y_pred_PDA_reg
y_pred_PDA_reg_copy[y_pred_PDA_reg_copy <0] =0
y_pred_PDA_reg_copy[y_pred_PDA_reg_copy >1] =1

y_pred_PDA_L1_copy = y_pred_PDA_L1
y_pred_PDA_L1_copy[y_pred_PDA_L1_copy <0] =0
y_pred_PDA_L1_copy[y_pred_PDA_L1_copy >1] =1


y_pred_PDA_L2_copy = y_pred_PDA_L2
y_pred_PDA_L2_copy[y_pred_PDA_L2_copy <0] =0
y_pred_PDA_L2_copy[y_pred_PDA_L2_copy >1] =1



#Revenue calculations for each prediction

#Optimal optimization value , just to verify balancing revenue function
P_DA_opt,obj_val_opt,p_E_up_list_opt,p_E_down_list_opt= Optimization_wind(wind_test,prices_test)
Revenue_opt, p_up_opt,p_down_opt = balancing_revenue(y_test_pda, wind_test, prices_test)


Revenue_reg, p_up_reg,p_down_reg = balancing_revenue(y_pred_PDA_reg_copy, wind_test, prices_test)

Revenue_L1, p_up_L1,p_down_L1 = balancing_revenue(y_pred_PDA_L1_copy, wind_test, prices_test)


Revenue_L2, p_up_L2,p_down_L2 = balancing_revenue(y_pred_PDA_L2_copy, wind_test, prices_test)


Revenue_rfr, p_up_rfr,p_down_rfr = balancing_revenue(y_pred_PDA_rfr, wind_test, prices_test)

Revenue_clust_rfr, p_up_clust_rfr,p_down_clust_rfr  =balancing_revenue(clust_rfr_pred,wind_test,prices_test)



#Plotting 
n_bins = 6

#Histogram plot 
fig, axs = plt.subplots(1, 6, sharey=True, tight_layout=True,figsize=(12, 4))
axs[0].hist(y_test_pda, bins=n_bins,color='lightblue', ec="black", lw=2)
axs[0].set(xticks=np.linspace(0,1,n_bins))
axs[0].set_title("Opt Model")

axs[1].hist(y_pred_PDA_reg, bins=n_bins,color='lightblue', ec="black", lw=2)
axs[1].set(xticks=np.linspace(0,1,n_bins))
axs[1].set_title("Regression")

axs[2].hist(y_pred_PDA_L1, bins=n_bins,color='lightblue', ec="black", lw=2)
axs[2].set(xticks=np.linspace(0,1,n_bins))
axs[2].set_title("L1 Regression")

axs[3].hist(y_pred_PDA_L2, bins=n_bins,color='lightblue', ec="black", lw=2)
axs[3].set(xticks=np.linspace(0,1,n_bins))
axs[3].set_title("L2 Regression")


axs[4].hist(y_pred_PDA_rfr, bins=n_bins,color='lightblue', ec="black", lw=2)
axs[4].set(xticks=np.linspace(0,1,n_bins))
axs[4].set_title("RFR Regression")

axs[5].hist(clust_rfr_pred, bins=n_bins,color='lightblue', ec="black", lw=2)
axs[5].set(xticks=np.linspace(0,1,n_bins))
axs[5].set_title("Clustered RFR Regression")
fig.suptitle('Distribution of Day Ahead Bids', fontsize=18)
fig.supylabel("Count of Day Ahead bids",fontsize=14)
fig.supxlabel("Value of Day Ahead bids",fontsize=14)

plt.show()

#True vs Pred plot 


fig, axs = plt.subplots(1, 5, sharey=True, tight_layout=True,figsize=(12, 4))
axs[0].scatter(y_test_pda,y_pred_PDA_reg, c='b', label='Fitted Data',alpha=0.4)
axs[0].plot([min(y_test_pda), max(y_test_pda)], [min(y_test_pda), max(y_test_pda)], 'k--', lw=2, label='Ideal Fit')
axs[0].set_title("Regression")

axs[1].scatter(y_test_pda,y_pred_PDA_L1, c='b', label='Fitted Data',alpha=0.4)
axs[1].plot([min(y_test_pda), max(y_test_pda)], [min(y_test_pda), max(y_test_pda)], 'k--', lw=2, label='Ideal Fit')
axs[1].set_title("L1 Regression")

axs[2].scatter(y_test_pda,y_pred_PDA_L2, c='b', label='Fitted Data',alpha=0.4)
axs[2].plot([min(y_test_pda), max(y_test_pda)], [min(y_test_pda), max(y_test_pda)], 'k--', lw=2, label='Ideal Fit')
axs[2].set_title("L2 Regression")

axs[3].scatter(y_test_pda,y_pred_PDA_rfr, c='b', label='Fitted Data',alpha=0.4)
axs[3].plot([min(y_test_pda), max(y_test_pda)], [min(y_test_pda), max(y_test_pda)], 'k--', lw=2, label='Ideal Fit')
axs[3].set_title("RFR Regression")

axs[4].scatter(y_test_pda,clust_rfr_pred, c='b', label='Fitted Data',alpha=0.4)
axs[4].plot([min(y_test_pda), max(y_test_pda)], [min(y_test_pda), max(y_test_pda)], 'k--', lw=2, label='Ideal Fit')
axs[4].set_title("RFR Regression")

handles, labels = axs[3].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper right')
fig.supxlabel("True Value", fontsize=14)
fig.supylabel("Predicted Value ",fontsize=14)
fig.suptitle('Prediction Vs True', fontsize=18)
plt.show()


Restricted license - for non-production use only - expires 2024-10-28


GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=538942fa-4593-4d1a-b90d-2d23669fe78c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>